In [6]:
!pip install langchain
!pip install tiktoken
!pip install pypdf
!pip install langchain-community
!pip install pypdf
!pip install langchain_google_genai
!pip install langchain_text_splitters
!pip install sentence-transformers

In [7]:
!pip install cohere

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.5/252.5 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 56.8 MB/s eta 0:00:00:00:01


In [8]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
# converting documents into smaller chunks of token
from langchain.text_splitter import RecursiveCharacterTextSplitter
# Embeddings
from langchain.embeddings import HuggingFaceEmbeddings
# Storing and creating a persitance vectorized form of text
from langchain_google_genai import GoogleGenerativeAI
from langchain_community.vectorstores import FAISS
import os
from langchain_community.llms import Cohere

import warnings
warnings.filterwarnings('ignore')

In [9]:
doc_loader = DirectoryLoader("/kaggle/input/physics-book/Physics_Book", glob = "./*.pdf", loader_cls = PyPDFLoader)

In [10]:
documents=doc_loader.load()

In [11]:
documents[0]

Document(metadata={'source': '/kaggle/input/physics-book/Physics_Book/jesc108.pdf', 'page': 0, 'page_label': '1'}, page_content='Science128\nFigure 8.1Figure 8.1Figure 8.1Figure 8.1Figure 8.1\nCreation of diversity over succeeding\ngenerations. The original organism at the top\nwill give rise to, say, two individuals, similar\nin body design, but with subtle differences.\nEach of them, in turn, will give rise to two\nindividuals in the next generation. Each of\nthe four individuals in the bottom row will\nbe differ\nent fr om each other . While some of\nthese differences will be unique, others will\nbe inherited from their respective parents,\nwho were different from each other .\nHeredity\n8CHAPTER\nW\ne have seen that reproductive processes give rise to new individuals\nthat are similar, but subtly different. We have discussed how some\namount of variation is produced even during asexual reproduction. And\nthe number of successful variations are maximised by the process of\nsexual re

In [12]:
all_texts = " ".join([doc.page_content for doc in documents])

In [13]:
len(all_texts)

480294

In [15]:
# splitting documents into smaller chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 400,
                                               chunk_overlap = 50,
                                               separators = ['\n', '\n\n', ' ', ''])
tokens = text_splitter.split_text(text = all_texts)

In [16]:
len(tokens)

1350

In [17]:
os.environ['HuggingFaceHub_API_Token']=""
os.environ['GOOGLE_API_KEY']=""

In [18]:
embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [20]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 67.0 MB/s eta 0:00:00:00:0100:01


In [21]:
# Indexing of data using FAISS

vector_store = FAISS.from_texts(tokens, embedding = embeddings)

In [22]:
retriever = vector_store.as_retriever(search_type = "similarity", search_kwargs = {"k":6})

In [23]:
retriever_docs = retriever.invoke("Give an example of a metal which is liquid ar room temperature.")

In [24]:
retriever_docs[0]

Document(id='1fd33c67-755d-497a-b819-889191e79f5b', metadata={}, page_content='or gases except bromine which is a liquid.\nDo non-metals also have physical properties similar to that of metals?\nLet us find out.\nActivity 3.6Activity 3.6Activity 3.6Activity 3.6Activity 3.6\n/square6Set up an electric circuit as shown in Fig. 3.2.\n/square6Place the metal to be tested in the circuit\nbetween terminals A and B as shown.\n/square6Does the bulb glow? What does this indicate?')

In [25]:
print(retriever_docs[0].page_content)

or gases except bromine which is a liquid.
Do non-metals also have physical properties similar to that of metals?
Let us find out.
Activity 3.6Activity 3.6Activity 3.6Activity 3.6Activity 3.6
/square6Set up an electric circuit as shown in Fig. 3.2.
/square6Place the metal to be tested in the circuit
between terminals A and B as shown.
/square6Does the bulb glow? What does this indicate?


In [26]:
#Cohere LLM for Retriever
prompt_template = """
Anwer the question as precise as possible using provided document. If answer is not available reply as 
"Answer is not available in input document." \n\n
Context: \n {context}?\n
Question: \n {question}?\n
Answer:
"""

In [27]:
os.environ['cohere_api_key'] = "jTIqO0PIhrTAV6OalQjp8U6MvhvrbAu6Y6aeZ67K"

In [28]:
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder

In [29]:
prompt = PromptTemplate.from_template(template = prompt_template)

In [30]:
def format_documents(document):
    return "\n\n".join(doc.page_content for doc in document)

In [31]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser


def generate_answer(question):
    cohere_llm = Cohere(model = "command", temperature = 0.1, cohere_api_key = os.getenv('cohere_api_key'))
    rag_chain = (
        {"context":retriever | format_documents, "question":RunnablePassthrough()}
        | prompt
        | cohere_llm
        | StrOutputParser()
    )
    return rag_chain.invoke(question)

In [32]:
generate_answer("What is symbol of iron.")

' The symbol of iron is "Fe". '

In [33]:
print(generate_answer("What is reflection and what is refraction?"))

 Reflection is the change in direction of light at a highly polished surface. The laws of reflection determine that the angle of incidence is equal to the angle of reflection, and that the incident ray, the normal to the mirror at the point of incidence and the reflected ray all lie in the same plane. 

Refraction is the change in direction of light as it passes from one medium to another. The laws of refraction determine that the ratio of the sine of the angle of incidence to the sine of the angle of refraction is equivalent to the ratio of the refractive indices of the two media. 

These principles are used in the construction and operation of mirrors and lenses, and in the study of optical phenomena in nature. 
The input document does not provide more information on these concepts, and thus, I can not provide additional information. 


In [34]:
print(generate_answer("What is air quality index of delhi?"))

 Answer is not available in input document.


In [35]:
generate_answer("Generate a summary about Magnetic Effects of Electic Current.")

" Magnetic Effects of Electric Current is about the history of electromagnetic concepts from the early experiments with moving charges and magnets by Hans Christian Oersted and André-Marie Ampère to the development of electromagnetic theory by James Clerk Maxwell and Heinrich Hertz's confirmation of Maxwell's theory through the discovery of radio waves.\n\nThe chapter also covers the importance of magnetism in Medicine and two methods of producing magnetic fields.\n\nUnfortunately, the provided document does not contain any summary section per se about the topic of magnetic effects of electric current, and therefore, I am unable to provide you with a concise summary of the chapter. "

In [36]:
print(generate_answer(" Why does a compass needle get deflected when brought near a bar magnet?"))

 A compass needle is deflected when brought near a bar magnet because the magnetic field from the magnet causes it to align in the direction of the field. This is why the needle points north when it is magnetically neutral, because the earth's magnetic field causes the needle to align that way. 


In [37]:
print(generate_answer("Explain Flemming's Left Hand rule."))

 Fleming's left-hand rule is a simple rule that illustrates the three directions of a magnetic field, current, and force. When you stretch the thumb, forefinger, and middle finger of your left hand such that they are mutually perpendicular, the rule states that the direction of the magnetic field is represented by the first finger, the direction of the current is represented by the second finger, and the direction of the force is represented by the thumb. The force is perpendicular to both the magnetic field and the current. 


In [38]:
print(generate_answer(""" Which of the following property of a proton can change while it moves
 freely in a magnetic field? (There may be more than one correct answer.)
 (a) mass
 (b) speed
 (c) velocity
 (d) momentum"""))

 The answer is (b) & (c) - Both speed and velocity of a proton can be changed while it moves freely in a magnetic field. The momentum of a proton is unlikely to change as it moves freely in a magnetic field. While the provided document does not explicitly mention these aspects, it focuses on electric circuits and domestic electricity, which are different topics entirely. 


In [39]:
print(generate_answer("""A positively-charged particle (alpha-particle) projected towards west is
 deflected towards north by a magnetic field. The direction of magnetic field is 
 (a) towards south
 (b) towards east
 (c) downward
 (d) upward."""))

 (b) towards east. The direction of magnetic field is determined by convention as the direction in which the north pole of a compass needle moves inside it. The magnetic field lines emerge from the north pole and merge at the south pole of the magnet. Therefore, the answer is option (b), where the magnetic field is towards the east, as the question asks for the direction in which a positively charged particle is deflected. 
